Ici on tente de faire un mapping entre les lieux de la db de corenting et l'API gouv

In [52]:
import pandas as pd
import numpy as np
import requests

On crée une liste des lieux à géocoder.
A chaque tentative, on supprime de la liste les éléments trouvés

In [53]:
# recuperaztion des lieux de corenting
session_data = pd.read_csv("../data/raw/session_planning_olympics.csv")
lieu_epreuves = set(session_data.lieu_epreuve)

In [54]:
print('longueur restantes', len(lieu_epreuves))

longueur restantes 40


In [55]:
# méthode directe
dataset = "data-es"
base_url = f"https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/{dataset}/records"
lieu_remove = []
for lieu in lieu_epreuves:
    params = {
        "where": f'nominstallation:"{lieu}"',
        "select": "numinstallation,nominstallation,adresse,codepostal,new_name,new_code,nomequipement,coordgpsx,coordgpsy, carac72"
    }
    res = requests.get(url=base_url, params=params, timeout=2)
    if res.json()['total_count'] > 0:
        print('trouvé')
        lieu_remove.append(lieu)
        pd.DataFrame(res.json()['results']).to_csv(f"../data/raw/{lieu}_basic.csv")
for lieu in lieu_remove:
    lieu_epreuves.remove(lieu)

trouvé
trouvé
trouvé


In [56]:
print('longueur restantes', len(lieu_epreuves))

longueur restantes 37


In [57]:
# méthode caps
dataset = "data-es"
base_url = f"https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/{dataset}/records"
lieu_remove = []
for lieu in lieu_epreuves:
    params = {
        "where": f'nominstallation:"{lieu.upper()}"',
        "select": "numinstallation,nominstallation,adresse,codepostal,new_name,new_code,nomequipement,coordgpsx,coordgpsy, carac72"
    }
    res = requests.get(url=base_url, params=params, timeout=2)
    if res.json()['total_count'] > 0:
        print('trouvé')
        lieu_remove.append(lieu)
        pd.DataFrame(res.json()['results']).to_csv(f"../data/raw/{lieu}_upper.csv")
for lieu in lieu_remove:
    lieu_epreuves.remove(lieu)
print('longueur restantes', len(lieu_epreuves))

trouvé
trouvé
trouvé
trouvé
longueur restantes 33


In [58]:
print('longueur restantes', len(lieu_epreuves))

longueur restantes 33


In [59]:
# méthode caps
dataset = "data-es"
base_url = f"https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/{dataset}/records"
lieu_remove = []
for lieu in lieu_epreuves:
    params = {
        "where": f'nominstallation:"{lieu.lower()}"',
        "select": "numinstallation,nominstallation,adresse,codepostal,new_name,new_code,nomequipement,coordgpsx,coordgpsy, carac72"
    }
    res = requests.get(url=base_url, params=params, timeout=2)
    if res.json()['total_count'] > 0:
        print('trouvé')
        lieu_remove.append(lieu)
        pd.DataFrame(res.json()['results']).to_csv(f"../data/raw/{lieu}_lower.csv")
for lieu in lieu_remove:
    lieu_epreuves.remove(lieu)

print('longueur restantes', len(lieu_epreuves))

longueur restantes 33
